# Configuration

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from pymatreader import read_mat
import json 

project_root = Path("/home/heberto/buzaki")
session_path = project_root /  "fCamk1_200827_sess9"
assert session_path.is_dir()

session_files_path_list = list(session_path.iterdir())
# Dump to a file in the same folder
json_directory = Path.cwd() / "_json_files"
json_directory.mkdir(exist_ok=True)


# Adjust the figure size
plt.figure(figsize=(16, 8))
# Adjust all the fonts
plt.rcParams.update({'font.size': 25})

# Some helper functions

In [ ]:
def format_files_as_string(files):
    # Add a tuple with size
    files = [(file_path.name, f"size: {file_path.stat().st_size / 1024**2 :,.2f} MiB") for file_path in files]

    # Sort the list by file size in descending order
    files.sort(key=lambda x: x[1], reverse=True)

    # Determine the maximum width for alignment based on longest file name length
    max_file_name_length = max(len(name) for name, _ in files)
    max_size_width = max_file_name_length + len("  size: ")

    # Create new list with formatted strings
    file_as_string = [f"{name.ljust(max_file_name_length)} {size}" for name, size in files]
    
    return file_as_string


def build_keys_and_types(dictionary):
    """
    Construct a new dictionary that contains type and optionally value or shape information 
    for each value in the input dictionary. This function handles nested dictionaries and treats 
    numpy arrays and lists in a specific way.

    The function creates a new key-value pair for every key-value pair in the input dictionary. 
    The value is another dictionary that always contains the 'type' key and might contain the 'value', 
    'shape' or 'length' key depending on the original value's type and size.

    Parameters
    ----------
    dictionary : dict
        The input dictionary to be analyzed. It may contain nested dictionaries, numpy arrays,
        lists, or other types of values.

    Returns
    -------
    output_dict : dict
        The output dictionary that provides detailed information about each value in the original dictionary.
        The value corresponding to each key is another dictionary that contains the following key-value pairs:
        - 'type': A string describing the type of the original value.
        - 'value': A string representing the original value. This key is present if the original value 
        is not a numpy array or list, or if it's a small numpy array or list (i.e., with fewer than 10 elements).
        - 'shape': A string representing the shape of the original numpy array. This key is present if the 
        original value is a numpy array with 10 or more elements.
    """
    output_dict = {}
    
    for key, value in dictionary.items():
        if isinstance(value, dict):
            output_dict[key] = build_keys_and_types(value)
        else:
            output_dict[key] = extract_value_info(value)

    return output_dict


def extract_value_info(value):
    """
    Generate a dictionary that contains type and optionally value or shape information about the input value.

    This function always adds the 'type' key to the output dictionary. Depending on the input value's type and size,
    it might also add the 'value', 'shape', or 'length' key.

    Parameters
    ----------
    value : object
        The input value. It can be of any type, including numpy array and list.

    Returns
    -------
    value_info : dict
        The output dictionary that contains the following key-value pairs:
        - 'type': A string describing the input value's type.
        - 'value': A string representing the input value. This key is present if the input value is not a numpy 
        array or list, or if it's a small numpy array or list (i.e., with fewer than 10 elements).
        - 'shape': A string representing the shape of the input value if it's a numpy array with 10 or more elements.
    """
    value_info = {"type": str(type(value))}
    
    if isinstance(value, (np.ndarray, list)):
        length = len(value) if isinstance(value, list) else value.size
        value_info["shape"] = str(value.shape) if isinstance(value, np.ndarray) else str(length)
        no_list_or_array_inside = not any(isinstance(item, (np.ndarray, list)) for item in value)
        if length < 10 and no_list_or_array_inside:
            value_info["value"] = str(value)
    else:
        value_info["value"] = str(value)

    return value_info

# Session overview

#### Mat files

In [ ]:
# Retrieve file paths and sizes

condition = lambda file_path: ".mat" in file_path.name
files = [file_path for file_path in session_files_path_list if condition(file_path)]

files_as_string = format_files_as_string(files)
files_as_string

### The rest

In [ ]:
condition = lambda file_path : ".mat" not in file_path.name and file_path.is_file()
files = [file_path for file_path in session_files_path_list if condition(file_path)]

files_as_string = format_files_as_string(files)
files_as_string


This is the output:

* `analogin.dat`, `8,203.18 MiB` : Probably raw data
* `fCamk1_200827_sess9.dat`, `32,812.73 MiB` : TODO
* `auxiliary.dat`, `3,076.19 MiB` : TODO
* `time.dat`, `2,050.80 MiB` :  TODO
* `TrialMapsAndRasters.pptx`, `1.02 MiB` : Collection of rasters, I don't think it matters. 
* `fCamk1_200827_sess9.lfp`, `1,367.20 MiB` :  LFP
* `digitalin.dat`, `1,025.40 MiB` :  TODO
* `supply.dat`, `1,025.40 MiB` :  TODO
* `pulTime.npy`, `0.32 MiB` : TODO
* `fCamk1_200827_sess9_HSE.HSE.evt`, `0.28 MiB` : TODO 
* `deepSuperficial_classification_fromRipples.png`, `0.14 MiB` : Figure, probably does not matter 
* `bz_DetectSWR.log`, `0.05 MiB` :  TODO
* `bz_DetectSWR_manu.log`, `0.05 MiB` : TODO
* `fCamk1_200827_sess9.xml`, `0.04 MiB` : TODO
* `fCamk1_200827_sess9.nrs`, `0.00 MiB` : TODO
* `4nA pulses.txt`, `0.00 MiB` : TODO
* `info.rhd`, `0.00 MiB` : TODO

#### Folders / Directories

In [ ]:
condition = lambda file_path : ".mat" not in file_path.name and file_path.is_file()
files = [file_path for file_path in session_files_path_list if condition(file_path)]

files_as_string = format_files_as_string(files)
files_as_string

# Explore specific files

## Sessions

### `session.mat`

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.session.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)

### `sessionInfo.mat`

In [ ]:

file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.sessionInfo.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)

### `.EventExplorer.SessionMetadata`

In [ ]:


file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.EventExplorer.SessionMetadata.mat"

assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)

This is empty

## Behavior 

### `Behavior.mat`

In [ ]:
file_path = session_path / f"{session_path.stem}.Behavior.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


### `behavior.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.behavior.cellinfo.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


I think this is the file with the most information. 

### `Tracking.Behavior.mat`

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.Tracking.Behavior.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
mat_file["tracking"]["position"]["x"][:5]

## Subfolder

### `Tracking.Behavior.mat` subfolder

In [ ]:
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / f"{sub_folder.stem}.Tracking.Behavior.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + "_sub_folder" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
mat_file["tracking"]["position"]["x"][:5]

It seems that this in the file above have the same position data

### `Linearized.Behavior.mat` subfolder

In [ ]:
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / f"{sub_folder.stem}.Linearized.Behavior.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + "_sub_folder" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
mat_file["behavior"]["position"]["x"][:5]

And it is the same position again here. 

However, this has the trials as well.

In [ ]:
mat_file["behavior"]["trials"]["startPoint"][:5]

This seems the position and not the times as we need. Let's look for other folder to see if we can find the trials

#### Camera matlab file

In [ ]:
file_path = session_files_path_list[0]
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / "Basler_acA1280-60gc__21606137__20200827_110730202.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "camera" + "_sub_folder" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)




Here they just copied the camera frame by frame (!)

```

{
  "frames": {
    "r": {
      "type": "<class 'numpy.ndarray'>",
      "shape": "(1024, 204, 53143)"
    }
  }
}
```
How long is the video 53_143 / 30 = 1_771.43 seconds -> 29.52 minutes

Taking the sampling_rate of 30 Hz, we can see that the video is 1_771.43 seconds long and the ntransform to minutes.

### Virtual maze mat

In [ ]:
file_path = session_files_path_list[0]
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / "virtualMaze.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = f"{file_path.stem}" + "_sub_folder" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


This seems just like a map of the maze

```
  "maze": {
    "type": "<class 'numpy.ndarray'>",
    "value": "[[ 11.09870461 109.25348571]\n [ 12.12486221   0.85756268]]"
  }
```

### Roi tracking .mat file

In [ ]:
file_path = session_files_path_list[0]
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / "roiTracking.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = f"{file_path.stem}" + "_sub_folder" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


I think this is a just a map with the reigon where they are doing the tracking of the position with the camera:

```
  "roiTracking": {
    "type": "<class 'numpy.ndarray'>",
    "value": "[[  31.25 1015.  ]\n [ 185.75 1019.5 ]\n [ 184.25    5.5 ]\n [  17.75    2.5 ]\n [  31.25 1015.  ]]"
  }
```

### Amplifier digital events `.mat`

In [ ]:
file_path = session_files_path_list[0]
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / "amplifier.DigitalIn.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = f"{file_path.stem}" + "_sub_folder" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
mat_file["digitalIn"].keys()

In [ ]:
mat_file["digitalIn"]["ints"][0].shape

In [ ]:
mat_file["digitalIn"]["ints"][0]  # I think those ar the intervals of the pulses in the local time for the behavior

In [ ]:
mat_file["digitalIn"]["dur"][1]  # I don't think they are ordered though

This is pulse duration

In [ ]:
mat_file["digitalIn"]["intsPeriods"]  # Note sure what it is this

#### TTL / Synch

```
dict_keys(['timestampsOn', 'timestampsOff', 'ints', 'dur', 'intsPeriods'])
```
I think this is the TTLs that synchronize the events. Come to this again.

In [ ]:
mat_file["digitalIn"]["timestampsOff"][0].shape

In [ ]:
mat_file["digitalIn"]["timestampsOff"][3].shape

In [ ]:
mat_file["digitalIn"]["dur"][0].shape

### Analog signals in sub-folder

We have the following
* `auxiliary.dat`
* `supply.dat`
* `time.dat`
* `digitalin.dat`

But two xml files:
* `fCamk1_200827_sess9.xml` which I guess it the main recorder
* `amplifier.xml` which I don't know what it does.

The first is on the top directory, the other one is here.

Note that in the sub-folder we only have `amplifier.xml` are they different?

Let's test this

In [ ]:
xml_file_path =  session_path /  "fCamk1_200827_sess9.xml"

from xml.etree import ElementTree

tree = ElementTree.parse(xml_file_path)
root = tree.getroot()
acq = root.find('acquisitionSystem')
nbits = int(acq.find('nBits').text)
num_channels = int(acq.find('nChannels').text)
sampling_rate = float(acq.find('samplingRate').text)
voltage_range = float(acq.find('voltageRange').text)
# offset = int(acq.find('offset').text)
amplification = float(acq.find('amplification').text)

print(f"{num_channels=}, {sampling_rate=}, {voltage_range=}, {amplification=}")


In [ ]:

sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()

xml_file_path =  sub_folder /  "amplifier.xml"

from xml.etree import ElementTree

tree = ElementTree.parse(xml_file_path)
root = tree.getroot()
acq = root.find('acquisitionSystem')
nbits = int(acq.find('nBits').text)
num_channels = int(acq.find('nChannels').text)
sampling_rate = float(acq.find('samplingRate').text)
voltage_range = float(acq.find('voltageRange').text)
# offset = int(acq.find('offset').text)
amplification = float(acq.find('amplification').text)

print(f"{num_channels=}, {sampling_rate=}, {voltage_range=}, {amplification=}")


Output:
The top level:
```
num_channels=32, sampling_rate=30000.0, voltage_range=20.0, amplification=1000.0
```

The amplifier:
```
num_channels=32, sampling_rate=30000.0, voltage_range=20.0, amplification=1000.0
```

They seem similar. I wonder what we have two but we have a lot of redundancy in the conversion data.

### Signals

In [ ]:
signal_names = ["digitalin.dat", "time.dat", "supply.dat", "auxiliary.dat"]


from spikeinterface.extractors.neoextractors import NeuroScopeRecordingExtractor

sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()

for name in signal_names:
    file_path = sub_folder / f"{name}" 
    assert file_path.is_file(), file_path

    sig_dtype = 'int16' if nbits <= 16 else 'int32'
    data = np.memmap(file_path, dtype=sig_dtype, mode='r', offset=0).reshape(-1, num_channels)
    time = data.shape[0] / sampling_rate
    num_samples = data.shape[0]

    print("---------------")
    print(f"{file_path.name=} \n")
    print(f"num_samples: {num_samples:,}, time: {time:.2f} seconds")

Output:
```
---------------
file_path.name='digitalin.dat' 

num_samples: 1,684,470, time: 56.15 seconds
---------------
file_path.name='time.dat' 

num_samples: 3,368,940, time: 112.30 seconds
---------------
file_path.name='supply.dat' 

num_samples: 1,684,470, time: 56.15 seconds
---------------
file_path.name='auxiliary.dat' 

num_samples: 5,053,410, time: 168.45 seconds
```

## `ripples.events.mat`

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.ripples.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)

In [ ]:
ripple_intervals = ripples_data["timestamps"]
peaks = ripples_data["peaks"]
peak_normed_power = ripples_data["peakNormedPower"]

ripple_stats_data = ripples_data["rippleStats"]["data"]
peak_frequency = ripple_stats_data["peakFrequency"]
peak_duration = ripple_stats_data["duration"]
peak_amplitude = ripple_stats_data["peakAmplitude"]

peaks.shape, peak_normed_power.shape, peak_frequency.shape, peak_duration.shape, peak_amplitude.shape

In [ ]:
# Extract indexed data
ripple_stats_maps = ripples_data["rippleStats"]["maps"]

ripple_raw = ripple_stats_maps["ripples_raw"]
ripple_frequency = ripple_stats_maps["frequency"]
ripple_phase = ripple_stats_maps["phase"]
ripple_amplitude = ripple_stats_maps["amplitude"]
ripple_raw.shape, ripple_frequency.shape, ripple_phase.shape, ripple_amplitude.shape

## Channels

### Channel map

In [ ]:
file_path = session_path / "chanMap.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "chanMap"+ ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
connected = mat_file["connected"]  

In [ ]:
mat_file["chanMap"]

In [ ]:
mat_file["chanMap0ind"]

## LED

In [ ]:
condition = lambda file_path : "LED" in file_path.name
led_files = [file_path for file_path in session_files_path_list if condition(file_path)] 

files_as_string = format_files_as_string(led_files)
files_as_string

In [ ]:
for file_path in led_files:
    if "uLEDResponse_raster" in file_path.name:
        continue
    print(file_path.name)
    assert file_path.is_file(), file_path

    mat_file = read_mat(file_path) 

    result = build_keys_and_types(mat_file)
    json_output = json.dumps(result, indent=2)

    json_name = "".join(file_path.suffixes)[1:] + ".json"
    with open(json_directory / json_name, 'w') as f:
        f.write(json_output)


## Optogenetic

In [ ]:
condition = lambda file_path : "optogenetic" in file_path.name
files = [file_path for file_path in session_files_path_list if condition(file_path)] 

files_as_string = format_files_as_string(files)
files_as_string

In [ ]:
file_path = session_path / f"{session_path.stem}.optogeneticPulses.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
file_path = session_path / f"{session_path.stem}.optogeneticResponse.cellinfo.mat"
assert file_path.is_file(), file_path

import h5py 

file_path = h5py.File(file_path, 'r')
# mat_file = read_mat(file_path) 
file_path["optogeneticResponses"].keys()


In [ ]:
file_path["optogeneticResponses"]["stimulationEpochs"][:][:, :2]

## EEG

In [ ]:
condition = lambda file_path : "eeg" in file_path.name
files = [file_path for file_path in session_files_path_list if condition(file_path)] 

files_as_string = format_files_as_string(files)
files_as_string



In [ ]:
file_path = session_path / f"{session_path.stem}.eegstates.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


## `SleepState.states`

In [ ]:
file_path = session_path / f"{session_path.stem}.SleepState.states.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


## EMG
It seems there is only one file

In [ ]:
file_path = session_path / f"{session_path.stem}.EMGFromLFP.LFP.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)

## Theta (`thetaEpochs.states`)

In [ ]:
file_path = session_path / f"{session_path.stem}.thetaEpochs.states.mat"
assert file_path.is_file(), file_path
mat_file = read_mat(file_path)

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


## `pulses.events.mat`

In [ ]:
file_path = session_path / f"{session_path.stem}.pulses.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
mat_file["pulses"].keys()

In [ ]:
mat_file["pulses"]["eventGroupID"][:][:5]

In [ ]:
mat_file["pulses"]["analogChannel"][:][:5]

## `ws_temp.mat`

In [ ]:
file_path = session_path / f"ws_temp.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "ws_temp" + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


## Analog signals in top folder

### Raw signal and LFP

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.dat"
assert file_path.is_file(), file_path

from spikeinterface.extractors.neoextractors import NeuroScopeRecordingExtractor

recording = NeuroScopeRecordingExtractor(file_path=file_path)
recording

Output:
```
NeuroScopeRecordingExtractor: 32 channels - 30.0kHz - 1 segments - 537,603,840 samples 
                              17,920.13s (4.98 hours) - int16 dtype - 32.04 GiB
  file_path: /home/heberto/buzaki/fCamk1_200827_sess9/fCamk1_200827_sess9.dat
```
We see this is around 5 hours of recording. 

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.lfp"
assert file_path.is_file(), file_path

from spikeinterface.extractors.neoextractors import NeuroScopeRecordingExtractor

recording = NeuroScopeRecordingExtractor(file_path=file_path)
recording

Here we see that the sampling rate is wrong. Let's test with the current value from the paper

In [ ]:
sampling_rate = 1.25 * 10**3 # 1.25 kHz
duration_seconds = recording.get_num_frames() / sampling_rate
duration_minutes = duration_seconds / 60.0
duration_hours = duration_minutes / 60.0
duration_hours

The output is five hours. This should be corrected

### Other signals in top folder
We have the following
* `analogin.dat`
* `auxiliary.dat`
* `supply.dat`
* `time.dat`
* `digitalin.dat`


Let's check the XML first from where their metadata comes from:

In [ ]:
xml_file_path =  session_path /  "fCamk1_200827_sess9.xml"

from xml.etree import ElementTree

tree = ElementTree.parse(xml_file_path)
root = tree.getroot()
acq = root.find('acquisitionSystem')
nbits = int(acq.find('nBits').text)
num_channels = int(acq.find('nChannels').text)
sampling_rate = float(acq.find('samplingRate').text)
voltage_range = float(acq.find('voltageRange').text)
# offset = int(acq.find('offset').text)
amplification = float(acq.find('amplification').text)

print(f"{num_channels=}, {sampling_rate=}, {voltage_range=}, {amplification=}")


In [ ]:
signal_names = ["digitalin.dat", "time.dat", "supply.dat", "auxiliary.dat", "analogin.dat"]


for name in signal_names:
    file_path = session_path / f"{name}" 
    assert file_path.is_file(), file_path

    sig_dtype = 'int16' if nbits <= 16 else 'int32'
    data = np.memmap(file_path, dtype=sig_dtype, mode='r', offset=0).reshape(-1, num_channels)
    time = data.shape[0] / sampling_rate
    num_samples = data.shape[0]

    print("---------------")
    print(f"{file_path.name=} \n")
    print(f"num_samples: {num_samples:,}, time: {time / 60.0:.2f} minutes")

Output:
They seem too short:
```
---------------
file_path.name='digitalin.dat' 

num_samples: 16,800,120, time: 9.33 minutes
---------------
file_path.name='time.dat' 

num_samples: 33,600,240, time: 18.67 minutes
---------------
file_path.name='supply.dat' 

num_samples: 16,800,120, time: 9.33 minutes
---------------
file_path.name='auxiliary.dat' 

num_samples: 50,400,360, time: 28.00 minutes
---------------
file_path.name='analogin.dat' 

num_samples: 134,400,960, time: 74.67 minutes
```

## Camera / Video

In [ ]:
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / "Basler_acA1280-60gc__21606137__20200827_110730202.avi"
assert file_path.is_file(), file_path

In [ ]:
file_path

In [ ]:
sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()
file_path = sub_folder / "Basler_acA1280-60gc__21606137__20200827_110730202.avi"
assert file_path.is_file(), file_path

# Use Pyav to get the video metadata
import av
container = av.open(str(file_path))

video = container.streams.video[0]

print("width", video.codec_context.width)
print("height", video.codec_context.height)
print("pixel format", video.codec_context.pix_fmt)
print("frame rate", video.codec_context.framerate)
print("bit rate", video.codec_context.bit_rate)
print("codec", video.codec_context.codec_tag)



In [ ]:
video.start_time, video.average_rate, video.duration, video.time_base

In [ ]:
from neuroconv.datainterfaces.behavior.video.video_utils import get_video_timestamps

timestamps_cv = get_video_timestamps(file_path)


In [ ]:
from neuroconv.datainterfaces.behavior.sleap.sleap_utils import extract_timestamps


timestamps_sleap = np.array(extract_timestamps(file_path))

In [ ]:
timestamps_cv.shape, timestamps_sleap.shape

In [ ]:
timestamps_cv[0], timestamps_sleap[0]

In [ ]:
1 / 30.0

In [ ]:
timestmaps_cv_shifted = timestamps_cv + 0.0333333333333333
np.isclose(timestmaps_cv_shifted, timestamps_sleap)

In [ ]:
np.allclose(timestmaps_cv_shifted, timestamps_sleap)

In [ ]:
import av

container = av.open(str(file_path))
stream = container.streams.video[0]

counter = 0
for frame in container.decode(stream):
    time_base = frame.time_base
    dts_time = float(frame.dts * time_base)
    pts_time = float(frame.pts * time_base)
    time = float(frame.time)
    print(f"{dts_time= }, {pts_time=}, {time=}")
    counter += 1
    
    if counter > 5:
        break


In [ ]:
counter = 0
for packet in container.demux(stream):
    print(f"{counter=}")
    print('Packet PTS:', packet.pts)
    print('Packet DTS:', packet.dts)
    print('Packet Duration:', packet.duration)
    print('Packet Timebase:', packet.time_base)
    
    counter += 1
    if counter > 5:
        break

In [ ]:
import av

# Open the video file
container = av.open(str(file_path))
stream = container.streams.video[0]

# Iterate over the first 10 packets in the video stream
for i, packet in enumerate(container.demux(stream)):
    print(f'Packet #{i + 1}:')
    print('PTS:', packet.pts)
    print('DTS:', packet.dts)
    print('Duration:', packet.duration)
    print('Timebase:', packet.time_base)
    print('\n')  # Print a newline for readability
    
    # Stop after 10 packets
    if i >= 9:
        break

# Epochs


In [ ]:
file_path = session_path / f"{session_path.stem}.session.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
epoch_list = mat_file["session"]["epochs"]

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

epoch_df = pd.DataFrame(epoch_list)
epoch_df.stopTime.max() / (3600)

This prints out five hours as expected.

In [ ]:
epoch_df["duration_seconds"] = (epoch_df.stopTime - epoch_df.startTime)
epoch_df["duration_hours"] = epoch_df.duration_seconds / (3600)
epoch_df

In [ ]:
epoch_df["duration_hours"].sum()

In [ ]:
print(epoch_df.to_markdown(index=False))

| name                 |   startTime |   stopTime | behavioralParadigm   | environment   | manipulation     |   duration_hours |
|:---------------------|------------:|-----------:|:---------------------|:--------------|:-----------------|-----------------:|
| fCamk1_200827_084028 |        0    |    5194.88 | BaselinePre          | Home cage     | None             |         1.44302  |
| fCamk1_200827_101538 |     5194.88 |    8284.64 | PreStim              | Home cage     | uLED random stim |         0.858267 |
| fCamk1_200827_110712 |     8284.64 |   10081.4  | Maze                 | Linear maze   | uLED random stim |         0.499102 |
| fCamk1_200827_113839 |    10081.4  |   11803.6  | PostStim             | Home cage     | uLED random stim |         0.478382 |
| fCamk1_200827_125535 |    11803.6  |   17920.1  | BaselinePost         | Home cage     | None             |         1.69904  |

1,796.77 seconds  
How does it relate to 53143 samples which is the samples for the timestamps in the trials table for the different behavioral files in the .mat files

53143 samples / 1796.77 seconds = 29.6 samples per second (Hz)

This is approximately 30 Hz whic fits wit the sampling rate of the camera.  


# Trials

## `beahvior.cellinfo.mat`

In [ ]:
file_path = session_path / f"{session_path.stem}.behavior.cellinfo.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)

trial_data = mat_file["behavior"]["trials"]


The startTime is are the trial intervals

In [ ]:
trial_intervals = trial_data["startPoint"]
start_time, stop_time = trial_intervals[:, 0], trial_intervals[:, 1]
start_time.shape, stop_time.shape


### Visited arm

In [ ]:
visted_arm = trial_data["visitedArm"]
visted_arm.shape

#### Recordings
Seems to have only one value

In [ ]:
mat_file["behavior"]["trials"]["recordings"].shape

In [ ]:
np.unique(mat_file["behavior"]["trials"]["recordings"])

#### Let's take a look at the maps

In [ ]:
len(mat_file["behavior"]["maps"])

In [ ]:
mat_file["behavior"]["maps"][0].shape, mat_file["behavior"]["maps"][1].shape

In [ ]:
mat_file["behavior"]["maps"][0][:10]

In [ ]:
mat_file["behavior"]["noStimulatedMaps"][0].shape

In [ ]:
mat_file["behavior"]["noStimulatedMaps"][0][:5]

In [ ]:
mat_file["behavior"]["stimulatedMaps"][0].shape

In [ ]:
mat_file["behavior"]["noStimulatedMapsLegend"]

In [ ]:
mat_file["behavior"]["stimulatedMapsLegend"]

In [ ]:
timestamps = mat_file["behavior"]["timestamps"]
timestamps[:5]

It seems taht the timestamps are already synchronized. That is, they start at their epoch

In [ ]:
trials_timestamps = mat_file["behavior"]["masks"]["trials"]

In [ ]:
np.unique(trials_timestamps)

This seems to be a map from timestamps [the index here] to trial. Let's confirm that using a figure. The trials with nan should represent the baseline. Let's change them by -1

In [ ]:
# Set np.nan to -1 in trial_timestamps
trials_to_plot = trials_timestamps.copy()
trials_to_plot[np.isnan(trials_to_plot)] = -20

import matplotlib.pyplot as plt
plt.plot(trials_to_plot)


I am wrong. I think there is sjust some delay

In [ ]:
# Extract the indexes where the trials change
trial_change_idx = np.nonzero(np.diff(trials_to_plot) != 0)[0]

In [ ]:
trial_change_idx

In [ ]:
start_time = timestamps[trial_change_idx]
start_time

It seems that the trials cover all the behavioral epoch.
However, we also know that there is no stimulation on all of those trials. I wonder where I can get the times for the behavioral epoch of which there should be 5. Baseline - stim 1 - baseline - stim 2 - baseline.

In [ ]:
trial_intervals = mat_file["behavior"]["trials"]["startPoint"]
start_time, stop_time = trial_intervals[:, 0], trial_intervals[:, 1]

# Calculate distance between stop_time and next start_time

inter_trial_distance = start_time[1:] - stop_time[:-1]

inter_trial_distance


In [ ]:
maps = np.concatenate(mat_file["behavior"]["maps"], axis=0)
trial_mask = mat_file["behavior"]["masks"]["trials"]
maps.shape

In [ ]:
trial_mask = mat_file["behavior"]["masks"]["trials"]
trial_index = 1
mask = trial_mask == trial_index
position = mat_file["behavior"]["position"]
x, y = position["x"], position["y"]
x_trial, y_trial = x[mask], y[mask]


x_no_norm = x * maps[:, 0]
y_no_norm = y * maps[:, 1]

# Create a color gradient based on the index
color_map = plt.cm.get_cmap('plasma')
colors = np.linspace(0, 1, len(x_no_norm[mask]))

# Plot with color gradient
plt.scatter(x_no_norm[mask], y_no_norm[mask], c=colors, cmap=color_map)

plt.colorbar()  # Add colorbar for reference


## `Behavior.mat`
This has the visited arm

In [ ]:
file_path = session_path / f"{session_path.stem}.Behavior.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

In [ ]:
mat_file["behavior"]["maps"][0].shape, mat_file["behavior"]["maps"][1].shape

In [ ]:
trial_direction = mat_file["behavior"]["masks"]["trialsDirection"]
trial_direction[:5]

In [ ]:
trials = mat_file["behavior"]["trials"]
trial_intervals = trials["startPoint"]
visited_arm = trials["visitedArm"]

visited_arm[:5], visited_arm.shape

In [ ]:
# Transform vistied_arm to 0 if value is 1 and to 1 otherwise
visited_arm_complement = np.where(visited_arm == 1, 0, 1)

np.where(visited_arm_complement != trial_direction)[0]

In [ ]:
visited_arm[110], trial_direction[110]

By looking at the plot below, it seems that this is the direction up and down no left and right.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np

trial_mask = mat_file["behavior"]["masks"]["trials"]


trials = [108, 109, 110]
trials = [5, 6, 7, 8, 9]

fig, axs = plt.subplots(1, len(trials), figsize=(20, 4))  # Create subplots with 1 row and 5 columns

for i, trial_index in enumerate(trials):
    mask = trial_mask == trial_index
    position = mat_file["behavior"]["position"]
    x, y = position["x"], position["y"]

    # Create a color gradient based on the index
    color_map = plt.cm.get_cmap('RdBu_r')
    colors = np.linspace(0, 1, len(x[mask]))

    # Plot with color gradient
    scatter = axs[i].scatter(x[mask], y[mask], c=colors, cmap=color_map)

    axs[i].set_title(f'Trial {trial_index}')  # Set title for each subplot
    axs[i].set_xlabel('X')  # Set x-axis label for each subplot
    axs[i].set_ylabel('Y')  # Set y-axis label for each subplot

    # Add a small colorbar to each plot
    divider = make_axes_locatable(axs[i])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(scatter, cax=cax)

plt.tight_layout()  # Adjust spacing between subplots
print("visited_arm", visited_arm[trials])
print("trial_direction", trial_direction[trials])

# Positions


I want to see if the positions in different files have different units. 

Which are the files 



In [ ]:
# In the top level of the session folder

top_level_with_position = ["behavior.cellinfo.mat", "Behavior.mat"]


In [ ]:
file_name = top_level_with_position[1]
for file_name in top_level_with_position:
    file_path = session_path / f"{session_path.stem}.{file_name}"
    assert file_path.is_file(), file_path

    mat_file = read_mat(file_path) 
    position = mat_file["behavior"]["position"]
    x, y = position["x"], position["y"]

    plt.plot(x, y)


Same position data here

In [ ]:
file_name = "Linearized.Behavior.mat"

sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()

file_path = sub_folder / f"{sub_folder.stem}.{file_name}"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

position = mat_file["behavior"]["position"]
x, y = position["x"], position["y"]

plt.plot(x, y)


In [ ]:
file_name  = "Tracking.Behavior.mat"

sub_folder = session_path / f"{session_path.stem.rsplit('_', 1)[0]}_110712"
assert sub_folder.is_dir()

file_path = sub_folder / f"{sub_folder.stem}.{file_name}"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 


position = mat_file["tracking"]["position"]
x, y = position["x"], position["y"]

plt.plot(x, y)

# Pulses

In [ ]:
file_path = session_path / f"{session_path.stem}.pulses.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
pulses = mat_file["pulses"]
pulses.keys()

In [ ]:
pulses["timestamps"]

In [ ]:
duration = pulses["timestamps"][:, 1] - pulses["timestamps"][:, 0]
duration[:5]

In [ ]:
pulses["analogChannel"][:5]

In [ ]:
pulses["analogChannelsList"][:5]

In [ ]:
pulses["intsPeriods"][:5]

In [ ]:
file_path = session_path / f"{session_path.stem}.session.mat"
assert file_path.is_file(), file_path
brain_regions_to_channels = read_mat(file_path)["session"]["brainRegions"]

# Invert the dictionary brain_regions_to_channels
channel_to_brain_region_dict = {}
for region, value in brain_regions_to_channels.items():
    channels = value["channels"]
    channel_to_brain_region_dict |= {channel: region for channel in channels}
    

brain_region = [channel_to_brain_region_dict[i] for i in pulses["analogChannel"]]
brain_region[:5]


In [ ]:
pulses["eventGroupID"][:5]

In [ ]:
inter_times = pulses["timestamps"][1:] - pulses["timestamps"][:-1]

In [ ]:
pulses["duration"][:5]

In [ ]:
np.unique(pulses["eventGroupID"])

In [ ]:
np.unique(pulses["analogChannel"])

In [ ]:
plt.hist(pulses["amplitude"])

In [ ]:
pulses["timestamps"].shape

In [ ]:
periods = pulses["intsPeriods"]
first_pulse = periods[0][0]
last_pulse = periods[-1][1]

first_pulse, last_pulse, (last_pulse - first_pulse) / 3600

In [ ]:
pulse_durations = periods[:, 1] - periods[:, 0] 
plt.hist(pulse_durations)

In [ ]:


x = periods[:, 0]
y = np.ones_like(x)
width = periods[:, 1] - periods[:, 0]
plt.bar(x=x, height=y, width=width, bottom=0, align='edge')

# Let's plot the epochs as well

file_path = session_path / f"{session_path.stem}.session.mat"
assert file_path.is_file(), file_path

mat_file_epochs = read_mat(file_path)
epoch_list = mat_file_epochs["session"]["epochs"]

start_time_list = [epoch["startTime"] for epoch in epoch_list[1:-1]]
epoch_name = [epoch["behavioralParadigm"] for epoch in epoch_list[1:-1]]

# Add an annotation for every epoch in start_time_list
for start_time in start_time_list:
    plt.axvline(start_time, color="red", linestyle="--", alpha=0.5, linewidth=5)
    
# Add the name in the top for each epoch in epoch_name
for start_time, name in zip(start_time_list, epoch_name):
    plt.text(start_time, 1.1, name, rotation=45, horizontalalignment="center")
    
plt.xlabel("Seconds")

# Remote frame in the y axis
plt.yticks([])

# Remote the frame
plt.box(False)

Juding by the time and duration the pulses cover the three epochs. The Prestim, the Maze and the PostStim.



In [ ]:
np.unique(pulses["analogChannel"])

In [ ]:
np.unique(pulses["eventGroupID"])

In [ ]:
np.unique(pulses["analogChannelsList"])

In [ ]:
channel = pulses["analogChannel"]
channel.shape

In [ ]:
pulse_duration_ms = pulses["duration"] * 1000
plt.hist(pulse_duration_ms)

This is correct duration of 20 milliseconds as stated in the paper.

In [ ]:
plt.hist(pulses["analogChannel"])

In [ ]:
plt.hist(pulses["analogChannelsList"])

## Optogenic pulses


'fCamk1_200827_sess9.optogeneticResponse.cellinfo.mat size: 375.40 MiB',
 'fCamk1_200827_sess9.optogeneticPulses.events.mat 

In [ ]:
file_path = session_path / f"{session_path.stem}.optogeneticPulses.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

json_name = "".join(file_path.suffixes)[1:] + ".json"
with open(json_directory / json_name, 'w') as f:
    f.write(json_output)


In [ ]:
epochs_stimulation = mat_file["optoPulses"]["stimulationEpochs"]

In [ ]:
epoch_length = epochs_stimulation[:, 1] - epochs_stimulation[:, 0]
epochs_stimulation[epoch_length < 0]

Some of them have negative length. I am confused about what this means.

In [ ]:
# Plot a bar for the periods[:, 0]

# Adjust the figure size
plt.figure(figsize=(16, 8))
# Adjust all the fonts
plt.rcParams.update({'font.size': 25})

x = epochs_stimulation[:, 0]
y = np.ones_like(x)
width = epochs_stimulation[:, 1] - epochs_stimulation[:, 0]
plt.bar(x=x, height=y, bottom=0, align='edge')

# Let's plot the epochs as well

file_path = session_path / f"{session_path.stem}.session.mat"
assert file_path.is_file(), file_path

mat_file_epochs = read_mat(file_path)
epoch_list = mat_file_epochs["session"]["epochs"]

start_time_list = [epoch["startTime"] for epoch in epoch_list[1:-1]]
epoch_name = [epoch["behavioralParadigm"] for epoch in epoch_list[1:-1]]

# Add an annotation for every epoch in start_time_list
for start_time in start_time_list:
    plt.axvline(start_time, color="red", linestyle="--", alpha=0.5, linewidth=5)
    
# Add the name in the top for each epoch in epoch_name
for start_time, name in zip(start_time_list, epoch_name):
    plt.text(start_time, 1.1, name, rotation=45, horizontalalignment="center")
    
plt.xlabel("Seconds")

# Remote frame in the y axis
plt.yticks([])

# Remote the frame
plt.box(False)

# Sleep states

In [ ]:
file_path = session_path / f"{session_path.stem}.SleepState.states.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)


In [ ]:
sleep_states_data = mat_file["SleepState"]
sleep_states_data.keys()

In [ ]:
intervals = sleep_states_data["ints"]
available_states = [str(key) for key in intervals.keys()]
available_states

In [ ]:
intervals["WAKEstate"]

In [ ]:
intervals["REMstate"]

In [ ]:
idx = sleep_states_data["idx"]
statenames = idx["statenames"]
statenames, len(statenames)

In [ ]:
np.unique(idx["states"])

This are just the timestamps and the indexes corresponding to the statenames

In [ ]:
available_states = [state for state in statenames if len(state) > 0]
available_states

In [ ]:
idx["theta_states"]

In [ ]:
np.unique(idx["theta_states"]["states"])

Here, 0 probabl ymeans the absence of signal but then there is theta vs non-theta.

# Theta 

In [ ]:
file_path = session_path / f"{session_path.stem}.thetaEpochs.states.mat"
assert file_path.is_file(), file_path
mat_file = read_mat(file_path)


dd

# Session general information (`sesion.mat`)

In [ ]:
file_path = session_path / f"{session_path.stem}.session.mat"
assert file_path.is_file(), file_path
mat_file = read_mat(file_path)


# Add epochs to nwbfile

In [ ]:
epoch_list = mat_file["session"]["epochs"]
epoch_list[0]

In [ ]:
from pynwb.testing.mock.file import mock_NWBFile

nwb_file = mock_NWBFile()

nwb_file.add_epoch_column(name="behavioral_paradigm", description="The behavioral paradigm of the epoch")
nwb_file.add_epoch_column(name="environment", description="The environment in the epoch")
nwb_file.add_epoch_column(name="manipulation", description="The stimulus in the epoch")


In [ ]:
for epoch in epoch_list:
    start_time = float(epoch["startTime"])
    stop_time = float(epoch["stopTime"])
    behavioral_paradigm = epoch["behavioralParadigm"]
    environment = epoch["environment"]
    manipulation = epoch["manipulation"]
    
    nwb_file.add_epoch(start_time=start_time, stop_time=stop_time, behavioral_paradigm=behavioral_paradigm, environment=environment, manipulation=manipulation)    

In [ ]:
nwb_file.epochs.to_dataframe()

# Camera

In [ ]:
videos_available = list(session_path.rglob('*.avi'))
assert len(videos_available) == 1, f"There should be one and only one video file {videos_avaialble}"  

In [ ]:
from neuroconv.datainterfaces import VideoInterface

interface = VideoInterface(file_paths=videos_available)
interface.get_metadata()

# Rewards as events

In [ ]:
file_path = session_path / f"{session_path.stem}.Behavior.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path) 

events_data = mat_file["behavior"]["events"]

In [ ]:
# Extract timestamps and create labels for rewards
reward_r_timestamps = events_data["rReward"]
reward_l_timestamps = events_data["lReward"]
label_reward_r = np.ones(reward_r_timestamps.shape[0], dtype=int)
label_reward_l = np.zeros(reward_l_timestamps.shape[0], dtype=int)

# Create a structure to concatenate timestamps and sort by them
reward_r = np.vstack((reward_r_timestamps, label_reward_r))
reward_l = np.vstack((reward_l_timestamps, label_reward_l))
rewards = np.concatenate((reward_r, reward_l), axis=1)

timestamps_both_rewards = rewards[0, :]
rewards = rewards[:, timestamps_both_rewards.argsort()]

In [ ]:
timestamps = rewards[0, :]
assert np.all(np.diff(timestamps) > 0)

In [ ]:
plt.plot(np.arange(rewards.shape[1]), rewards[0, :])
# Add an identity line in the plot
np.diff(r)

In [ ]:
from ndx_events import LabeledEvents

timestamps = rewards[0, :]
data = rewards[1, :]

events = LabeledEvents(
    name='rewards',
    description='rewards in the linear track',
    timestamps=timestamps,
    data=data,
    labels=['right_reward', 'left_reward']
)

from pynwb.testing.mock.file import mock_NWBFile

nwbfile = mock_NWBFile()


nwbfile.add_acquisition(events)

# Add pulses as time intervals

In [ ]:
file_path = session_path / f"{session_path.stem}.pulses.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
pulses_data = mat_file["pulses"]
pulses_data.keys()

In [ ]:
pulse_intervals = pulses_data["timestamps"] 
electrode_channel = pulses_data["analogChannel"]
amplitude = pulses_data["amplitude"]



In [ ]:
from pynwb.testing.mock.file import mock_NWBFile

nwbfile = mock_NWBFile()

from pynwb.epoch import TimeIntervals

laser_description = (
"""
μLEDs were controlled with current (2-4.5 μA generating 0.02-0.1μW of total light power;
ref (15)) provided by a 12-channel current generator (OSC1Lite, NeuroNex Michigan Hub)
driven by an Arduino (https://github.com/valegarman), which delivered trapezoid (1ms rise time)
blue light (centered emission at 460 nm, emission surface area = 150 mm2) 20 ms pulses at
random sites with a randomly variable (40-60ms) offset.
"""
)

stimuli_laser_pulses = TimeIntervals(
    name="laser_pulses",
    description="intervals for each pulse stimuli for the uLED electrodes",
)

stimuli_laser_pulses.add_column(name="electrode_channel", description="The electrode channel for the pulse")
stimuli_laser_pulses.add_column(name="amplitude", description="The amplitude of the pulse")

for interval, channel, amp in zip(pulse_intervals, electrode_channel, amplitude):
    start_time, stop_time = interval
    channel = channel
    row_dict = {"start_time": start_time, "stop_time": stop_time, "electrode_channel": channel, "amplitude": amp}
    stimuli_laser_pulses.add_row(**row_dict)
    

nwbfile.add_time_intervals(stimuli_laser_pulses)

# Ripples as events

In [ ]:
file_path = session_files_path_list[0]
file_path = session_path / f"{session_path.stem}.ripples.events.mat"
assert file_path.is_file(), file_path

mat_file = read_mat(file_path)
ripples_data = mat_file["ripples"]   

In [ ]:
ripple_intervals = ripples_data["timestamps"]

peaks = ripples_data["peaks"]
peak_normed_power = ripples_data["peakNormedPower"]

ripple_stats_data = ripples_data["rippleStats"]["data"]

peak_frequencies = ripple_stats_data["peakFrequency"]
ripple_durations = ripple_stats_data["duration"]
peak_amplitudes = ripple_stats_data["peakAmplitude"]

descriptions = dict(
    ripple_durations="Duration of the ripple event.",
    peaks="Peak of the ripple.",
    peak_normed_power="Normed power of the peak.",
    peak_frequencies="Peak frequency of the ripple.",
    peak_amplitudes="Peak amplitude of the ripple.",
)



In [ ]:
from pynwb.testing.mock.file import mock_NWBFile
from pynwb import NWBFile, H5DataIO
from pynwb.epoch import TimeIntervals


nwbfile = mock_NWBFile()
name = "ripples"
table = TimeIntervals(name=name, description="Ripples and their metrics")


for start_time, stop_time in ripple_intervals:
    table.add_row(start_time=start_time, stop_time=stop_time)

for column_name, column_data in zip(
    list(descriptions), [ripple_durations, peaks, peak_normed_power, peak_frequencies, peak_amplitudes]
):
    table.add_column(
        name=column_name,
        description=descriptions[column_name],
        data=H5DataIO(column_data, compression="gzip"),
    )

# Extract indexed data

ripple_stats_maps = ripples_data["rippleStats"]["maps"]

ripple_raw = ripple_stats_maps["ripples_raw"]
ripple_frequencies = ripple_stats_maps["frequency"]
ripple_phases = ripple_stats_maps["phase"]
ripple_amplitudes = ripple_stats_maps["amplitude"]

indexed_descriptions = dict(
    ripple_raw="Extracted ripple data.",
    ripple_frequencies="Frequency of each point on the ripple.",
    ripple_phases="Phase of each point on the ripple.",
    ripple_amplitudes="Amplitude of each point on the ripple.",
)

for column_name, column_data in zip(
    list(indexed_descriptions), [ripple_raw, ripple_frequencies, ripple_phases, ripple_amplitudes]
):
    table.add_column(
        name=column_name,
        description=indexed_descriptions[column_name],
        index=list(range(column_data.shape[0])),
        data=H5DataIO(column_data, compression="gzip"),
    )
    



In [ ]:
ripple_frequencies.shape

# Compare sorting and sorting times

In [ ]:
from neuroconv.datainterfaces import CellExplorerSortingInterface

In [ ]:
file_path = session_path / f"{session_path.stem}.spikes.cellinfo.mat"

cell_explorer_interface = CellExplorerSortingInterface(file_path=file_path)

In [ ]:
sorter_cell_explorer = cell_explorer_interface.sorting_extractor
sorter_cell_explorer

In [ ]:
import pandas as pd
sorter = sorter_cell_explorer

spike_range = {}
for unit_id in sorter.unit_ids:

    spikes = sorter.get_unit_spike_train(unit_id=unit_id, return_times=True) / 3600
    first_spike, last_spike = spikes[0], spikes[-1]
    spike_range[unit_id] = dict(first_spike=first_spike, last_spike=last_spike)
    
pd.DataFrame(spike_range).max(axis=1)


In [ ]:
from neuroconv.datainterfaces import KiloSortSortingInterface

folder_path = session_path / "Kilosort_2020-08-28_231022"
kilosort_interface = KiloSortSortingInterface(folder_path=folder_path)
kilosort_sorter = kilosort_interface.sorting_extractor
kilosort_sorter

In [ ]:
import pandas as pd
sorter = kilosort_sorter

spike_range = {}
for unit_id in sorter.unit_ids:

    spikes = sorter.get_unit_spike_train(unit_id=unit_id, return_times=True) / 3600
    first_spike, last_spike = spikes[0], spikes[-1]
    spike_range[unit_id] = dict(first_spike=first_spike, last_spike=last_spike)
    
pd.DataFrame(spike_range).max(axis=1)


In [ ]:
folder_path = session_path / "Kilosort_2020-08-28_231022"
neuroscope_interface = NeuroScopeSortingInterface(folder_path=folder_path)

neuroscope_sorter = neuroscope_interface.sorting_extractor
neuroscope_sorter

# Check NWB result

In [ ]:
output_dir_path = Path.home() / "conversion_nwb"

stub_path_list = list((output_dir_path / "nwb_stub").iterdir())


In [ ]:
file_path = stub_path_list[0]

import pynwb 

# Open file with pynwb
nwbfile = pynwb.NWBHDF5IO(str(file_path), mode='r', load_namespaces=True).read()
nwbfile